In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load combined data
results = pd.read_csv('../data/dummy_data_combined.csv')
# Convert time to datetime
results.charttime = pd.to_datetime(results.charttime)
results.intime = pd.to_datetime(results.intime)
# Sort dataframe by stay id, item id and chart time
results.sort_values(by=['stay_id', 'itemid', 'charttime'], inplace=True)
# Reset index
results.reset_index(drop=True, inplace=True)

print(results.shape)
results.head()

(500, 7)


,stay_id,value,itemid,charttime,intime,age,sex
0,100,42.404009,1,2015-01-05 08:03:11,2015-01-03 21:14:53,26,M
1,100,88.624171,1,2015-01-05 15:09:39,2015-01-03 21:14:53,26,M
2,100,15.674791,1,2015-01-05 15:41:03,2015-01-03 21:14:53,26,M
3,100,73.266732,1,2015-01-07 13:10:46,2015-01-03 21:14:53,26,M
4,100,83.675556,3,2015-01-09 06:01:50,2015-01-03 21:14:53,26,M


In [3]:
# # Load admission data
# adm = pd.read_csv('../data/dummy_admissions.csv')
# # Convert time to datetime
# adm.intime = pd.to_datetime(adm.intime)
# # Sort dataframe by stay id
# adm.sort_values(by='stay_id', inplace=True)
# # Reset index
# adm.reset_index(drop=True, inplace=True)

# print(adm.shape)
# adm.head()

# Calculate $\Delta t$
### Calculate the time difference between two subsequent tests for each variable

In [4]:
def timedelta2hours(td):
    return td.days * 24 + td.seconds / 3600

In [5]:
results['delta_t'] = results.groupby(['stay_id', 'itemid']).charttime.diff()
results.delta_t = results.delta_t.apply(timedelta2hours)
results.delta_t

0            NaN
1       7.107778
2       0.523333
3      45.495278
4            NaN
         ...    
495    31.631389
496          NaN
497    31.323056
498    44.390833
499    38.455000
Name: delta_t, Length: 500, dtype: float64

In [6]:
results.tail()

,stay_id,value,itemid,charttime,intime,age,sex,delta_t
495,199,36.613274,4,2015-01-08 10:27:02,2015-01-01 02:52:37,56,M,31.631389
496,199,36.397021,5,2015-01-05 04:22:11,2015-01-01 02:52:37,56,M,NaN
497,199,48.778638,5,2015-01-06 11:41:34,2015-01-01 02:52:37,56,M,31.323056
498,199,32.914245,5,2015-01-08 08:05:01,2015-01-01 02:52:37,56,M,44.390833
499,199,24.839532,5,2015-01-09 22:32:19,2015-01-01 02:52:37,56,M,38.455000


### Calculate the time difference between the ICU admission and the first test for each var

In [7]:
def calculate_first_diff(x):
    return timedelta2hours(x.charttime - x.intime)

results.loc[results.delta_t.isna(), 'delta_t'] = results[results.delta_t.isna()].apply(calculate_first_diff, axis=1)    

In [8]:
results.tail()

,stay_id,value,itemid,charttime,intime,age,sex,delta_t
495,199,36.613274,4,2015-01-08 10:27:02,2015-01-01 02:52:37,56,M,31.631389
496,199,36.397021,5,2015-01-05 04:22:11,2015-01-01 02:52:37,56,M,97.492778
497,199,48.778638,5,2015-01-06 11:41:34,2015-01-01 02:52:37,56,M,31.323056
498,199,32.914245,5,2015-01-08 08:05:01,2015-01-01 02:52:37,56,M,44.390833
499,199,24.839532,5,2015-01-09 22:32:19,2015-01-01 02:52:37,56,M,38.455000


___
# Convert to GRU format
For each stay_id we create a table with itemid.nunique() rows and